In [11]:
import pandas as pd
import numpy as np

# 예시 데이터프레임 (1번 csv 가정)
for i in range(1,9):
    df = pd.read_csv('./data/'+str(i)+'호선_혼잡도_전체예측결과.csv')

    # 그룹핑 기준 컬럼
    group_cols = ['역명', '방향', '평일/주말', '시간대']

    # 칸번호 순정렬 보장
    df = df.sort_values(by=group_cols + ['칸번호'])

    # 각 그룹별로 '예측혼잡도' 리스트로 묶기
    grouped = df.groupby(group_cols)['예측혼잡도'].apply(list).reset_index()

    # 컬럼명 변경/신규 추가 (예시 - 필요에 따라 조정)
    grouped['congestionCar'] = grouped['예측혼잡도'].apply(lambda x: str([round(int(xx / 6), 1) for xx in x]))
    grouped['호선'] = '1호선'   # 필요시 일괄 추가
    grouped['상하행'] = grouped['방향'].map({'상행': 0, '하행': 1})
    # rouped['요일'] = 'MON' if grouped['평일/주말'].iloc[0] == '평일' else 'SAT' # 예시변환
    grouped['요일'] = grouped['평일/주말'].map({'평일': 'MON', '주말': 'SAT'})
    grouped['시간'] = grouped['시간대'].str[:2]
    grouped['분'] = grouped['시간대'].str[3:]
    grouped['요일(한글)'] = '월요일' if grouped['평일/주말'].iloc[0] == '평일' else '토요일' # 예시

    # 최종 원하는 포맷만 추출
    output = grouped[['호선','역명','요일','시간','분','상하행','congestionCar','요일(한글)']]

    # 저장/출력 등
    output.to_csv('./data/'+str(i)+'호선_혼잡도_전체예측결과_수정본.csv', index=False,encoding="euc-kr")
    print(output.head())


    호선   역명   요일  시간   분  상하행                             congestionCar 요일(한글)
0  1호선  동대문  SAT  06  00    0  [12, 12, 12, 12, 12, 12, 12, 12, 13, 12]    토요일
1  1호선  동대문  SAT  07  00    0  [15, 13, 13, 13, 13, 13, 13, 13, 15, 15]    토요일
2  1호선  동대문  SAT  08  00    0  [24, 22, 21, 22, 22, 22, 22, 22, 28, 24]    토요일
3  1호선  동대문  SAT  09  00    0  [24, 24, 24, 24, 24, 24, 24, 24, 27, 24]    토요일
4  1호선  동대문  SAT  10  00    0  [26, 28, 28, 28, 28, 28, 28, 28, 32, 26]    토요일
    호선  역명   요일  시간   분  상하행                             congestionCar 요일(한글)
0  1호선  강남  SAT  06  00    0  [11, 11, 11, 11, 11, 11, 11, 11, 12, 11]    토요일
1  1호선  강남  SAT  07  00    0  [11, 10, 10, 10, 10, 10, 10, 10, 11, 11]    토요일
2  1호선  강남  SAT  08  00    0  [20, 17, 17, 17, 17, 17, 17, 17, 18, 20]    토요일
3  1호선  강남  SAT  09  00    0  [22, 21, 21, 22, 22, 22, 22, 22, 22, 22]    토요일
4  1호선  강남  SAT  10  00    0  [24, 23, 23, 23, 24, 24, 24, 24, 24, 24]    토요일
    호선    역명   요일  시간   분  상하행                         con

In [ ]:
import pandas as pd
import numpy as np

# 1번: 예측 데이터 (각 칸별 row)
df1 = pd.read_csv('./data/1호선_혼잡도_전체예측결과.csv')
# 2번: wide 포맷(마스터 역할, 메타정보 제공)
df2 = pd.read_csv('../train/output/1호선_월요일_혼잡도_거점역.csv')

# 기본적으로 그룹핑 컬럼 추출
group_cols = ['역명', '방향', '평일/주말', '시간대']

# 1-1. 각 그룹의 값을 1에서 모으고, 칸별 예측혼잡도 리스트로
df1 = df1.sort_values(group_cols + ['칸번호'])  # 칸번호 순 정렬
grouped = df1.groupby(group_cols).agg({
    '예측혼잡도': lambda x: list(x),
    '혼잡레벨': 'first'  # 필요시
}).reset_index()

# 2번에서 key가 겹치는 행을 찾는다
def find_reference_row(row):
    # mapping: 방향→상하행, 평일/주말→요일 등
    ref = df2[
        (df2['역명'] == row['역명']) &
        (df2['시간'] == int(row['시간대'][:2])) &     # 시간 비교
        (df2['분'] == int(row['시간대'][3:])) &      # 분 비교
        (df2['상하행'] == (0 if row['방향']=='상행' else 1)) &
        (df2['요일(한글)'].str.startswith(row['평일/주말'][0]))  # "평일" → "월", "주말" → "토" 등 가정
    ]
    return ref.iloc[0] if len(ref) > 0 else pd.Series()

# 2-1. 2의 값으로 메타정보를 보충하며, 1의 혼잡도만 사용
result_rows = []
for idx, row in grouped.iterrows():
    ref = find_reference_row(row)
    out = {}
    # 2의 메타정보를 모두 복사
    if not ref.empty:
        out = ref.to_dict()
    # 1의 정보로 overwrite(예측 혼잡도)
    out.update({
        '예측congestionCar': str([round(x,1) for x in row['예측혼잡도']]),
        # 혹시 예측 혼잡레벨 등도 추가할 수 있음
        '혼잡레벨': row['혼잡레벨'],
    })
    result_rows.append(out)

# DataFrame 만들기
output = pd.DataFrame(result_rows)

# 필요한 컬럼만 최종 추출/정렬
want_cols = [
    '호선','역명','역코드','요일','시간','분','상하행','직통여부',
    '구간시작역코드','구간시작역','구간종료역코드','구간종료역',
    '직전역코드','직전역',
    '예측congestionCar','혼잡레벨',
    'congestionCar','요일(한글)'
]
present_cols = [c for c in want_cols if c in output.columns]
output = output[present_cols]

# 결과 저장/출력
output.to_csv('./data/1호선_혼잡도_전체예측결과_수정본2.csv', index=False)
print(output.head())


In [12]:
import pandas as pd

def modify_station_names_and_save(input_path, output_path):
    # 엑셀 파일 불러오기
    df = pd.read_csv(input_path,encoding='euc-kr')

    # '역명' 컬럼에 '역' 붙이기 (이미 '역'이 붙어 있으면 그대로)
    df['역명'] = df['역명'].apply(lambda x: x if str(x).endswith('역') else str(x) + '역')

    # 수정된 DataFrame을 새 엑셀 파일로 저장
    df.to_csv(output_path, index=False,encoding='euc-kr')

for i in range(1,9):
    modify_station_names_and_save('./data/'+str(i)+'호선_혼잡도_전체예측결과_수정본.csv',
                                  './data/result/'+str(i)+'호선_혼잡도_전체예측결과.csv')
